In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


# **LIBRARY**

In [5]:
import pandas as pd
import string

from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from IPython.display import display

In [6]:
df = pd.read_csv("/content/gdrive/MyDrive/pizza/orders.csv")
df_odetails = pd.read_csv("/content/gdrive/MyDrive/pizza/order_details.csv")
df_piin4 = pd.read_csv("/content/gdrive/MyDrive/pizza/pizzas.csv")
df_pitype = pd.read_csv("/content/gdrive/MyDrive/pizza/pizza_types.csv", encoding = 'Windows-1252')

In [ ]:
df.head()

,order_id,date,time,customer_id,total_payment
0,1,2015-01-01,11:38:36,742,13.25
1,2,2015-01-01,11:57:40,5819,92.00
2,3,2015-01-01,12:12:28,5250,37.25
3,4,2015-01-01,12:16:31,6608,16.50
4,5,2015-01-01,12:21:30,1375,16.50


# **RECOMMEND SYSTEM**

In [7]:
full_order_df = pd.merge(df, df_odetails, on='order_id', how='inner')
colab_df = pd.merge(full_order_df, df_piin4, on='pizza_id', how='inner')
colab_df = pd.merge(colab_df, df_pitype, on='pizza_type_id', how='inner')

colab_df.head()

,order_id,date,time,customer_id,total_payment,order_details_id,pizza_id,quantity,pizza_type_id,size,price,name,category,ingredients
0,1,2015-01-01,11:38:36,742,13.25,1,hawaiian_m,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,2,2015-01-01,11:57:40,5819,92.00,2,classic_dlx_m,1,classic_dlx,M,16.00,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
2,2,2015-01-01,11:57:40,5819,92.00,3,five_cheese_l,1,five_cheese,L,18.50,The Five Cheese Pizza,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go..."
3,2,2015-01-01,11:57:40,5819,92.00,4,ital_supr_l,1,ital_supr,L,20.75,The Italian Supreme Pizza,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni..."
4,2,2015-01-01,11:57:40,5819,92.00,5,mexicana_m,1,mexicana,M,16.00,The Mexicana Pizza,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O..."


# **CONTENT-BASED RECOMMENDERS**

**RECOMMENDATION BASED ON PIZZA CATEGORY AND INGREDIENTS**

In [8]:
df_cbrecommend = df_pitype['category'] + ' ' + df_pitype['ingredients']

**TEXT PREPROCESSING**

In [9]:
def clean_text(text):
    if isinstance(text, str):
        words = text.lower().split()
        words = [word.translate(str.maketrans('', '', string.punctuation)) for word in words]
        processed_text = ' '.join(words)
        return processed_text
    return text
df_cbrecommend = df_cbrecommend.apply(clean_text)


In [10]:
tokenizer = TweetTokenizer()
df_cbrecommend = df_cbrecommend.apply(lambda x: tokenizer.tokenize(x))

In [11]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words='english', lowercase=False, token_pattern=None)
X = tfidf.fit_transform(df_cbrecommend)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


**COSINE SIMILARITY**

In [12]:
cosine_sim = cosine_similarity(X, X)

df_recommend_cb = df_pitype.copy()
df_recommend_cb['id'] = df_recommend_cb.index
df_recommend_cb['piz_name'] = df_recommend_cb['name'].str.lower()

In [13]:
def recommend_cb(name, cosine_sim=cosine_sim):
    try:
        piz_name = name.strip().lower()
        if piz_name not in df_recommend_cb['piz_name'].values:
            return f"The dish '{name}' is not on the menu."
        idx = df_recommend_cb.loc[df_recommend_cb['piz_name'] == piz_name].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top10 = [i[0] for i in sim_scores[1:16]]
        recommend_content = df_recommend_cb[['name', 'ingredients']].iloc[top10].reset_index(drop=True)
        print(f"Similar dishes to '{name}', you might like:")
        return recommend_content
    except Exception as e:
        return f"Error: {str(e)}"

recommend_cb('The Barbecue Chicken Pizza')

Similar dishes to 'The Barbecue Chicken Pizza', you might like:


,name,ingredients
0,The Chicken Pesto Pizza,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
1,The Southwest Chicken Pizza,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."
2,The Chicken Alfredo Pizza,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha..."
4,The Vegetables + Vegetables Pizza,"Mushrooms, Tomatoes, Red Peppers, Green Pepper..."
5,The Mexicana Pizza,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O..."
6,The California Chicken Pizza,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
7,The Classic Deluxe Pizza,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
8,The Napolitana Pizza,"Tomatoes, Anchovies, Green Olives, Red Onions,..."
9,The Italian Vegetables Pizza,"Eggplant, Artichokes, Tomatoes, Zucchini, Red ..."


# **USER-BASED COLLABORATIVE FILTERING**




In [14]:
user_item_matrix = colab_df.pivot_table(index='order_id', columns='pizza_type_id', values='quantity', fill_value=0)
user_item_sparse = csr_matrix(user_item_matrix.values)
cosine_sim_colab = cosine_similarity(user_item_sparse, user_item_sparse)

In [39]:
def recommend_pizza(order_id, cosine_sim_colab, user_item_matrix, colab_df, top_n=3):
    order_idx = user_item_matrix.index.get_loc(order_id)
    sim_scores = list(enumerate(cosine_sim_colab[order_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_orders = [i[0] for i in sim_scores[1:top_n+1]]

    top_order_pizza_types = user_item_matrix.loc[top_orders].stack().reset_index()[['order_id', 'pizza_type_id']]
    top_order_pizza_types = top_order_pizza_types[top_order_pizza_types['pizza_type_id'] != 0]

    pizza_details = pd.merge(top_order_pizza_types, colab_df[['order_id', 'ingredients', 'name', 'pizza_type_id']], on=['order_id'], how='left')
    pizza_details = pizza_details.drop_duplicates(subset=['name'])

    recommendations = pizza_details[['name', 'ingredients']].reset_index(drop=True)

    return recommendations

recommend_pizza(order_id=25, cosine_sim_colab=cosine_sim_colab, user_item_matrix=user_item_matrix, colab_df=colab_df, top_n=3)

,name,ingredients
0,The Soppressata Pizza,"Soppressata Salami, Fontina Cheese, Mozzarella..."
1,The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha..."
2,The Prosciutto and Arugula Pizza,"Prosciutto di San Daniele, Arugula, Mozzarella..."
3,The Big Meat Pizza,"Bacon, Pepperoni, Italian Sausage, Chorizo Sau..."
4,The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Go..."


# **HYBRID RECOMMENDER SYSTEM FOR ORDERING**




**NEW CUSTOMER**

In [17]:
def new_customer():
    recommendation_history = pd.DataFrame(columns=['name', 'ingredients'])

    while True:
        name = input("Enter the dish name (or enter 'done' to finish, enter 'more' for more suggestions): ").strip()

        if name.lower() == 'done':
            print("Enjoy your meal.")
            return recommendation_history

        elif name.lower() == 'more':
            all_pizzas = df_pitype[['name', 'ingredients']]
            available_pizzas = all_pizzas[~all_pizzas['name'].isin(recommendation_history['name'])]
            new_recommendations = available_pizzas
            recommendation_history = pd.concat([recommendation_history, new_recommendations], ignore_index=True)
            return recommendation_history

        else:
            recommendations = recommend_cb(name)

            if isinstance(recommendations, pd.DataFrame):
                recommendations = recommendations[~recommendations['name'].isin(recommendation_history['name'])]

                if recommendations.empty:
                    all_pizzas = df_pitype[['name', 'ingredients']]
                    available_pizzas = all_pizzas[~all_pizzas['name'].isin(recommendation_history['name'])]

                    if available_pizzas.empty:
                        print("There are no more pizza dishes to suggest.")
                    else:
                        new_recommendations = available_pizzas
                        recommendation_history = pd.concat([recommendation_history, new_recommendations], ignore_index=True)

                else:
                    recommendation_history = pd.concat([recommendation_history, recommendations], ignore_index=True)
                    print("\nRecommendation history in this session:")
                    print(recommendation_history)

            else:
                print(recommendations)

**OLD CUSTOMER**

In [34]:
def old_customer(new_order):
    customer_orders = colab_df[colab_df['customer_id'] == int(new_order)]
    if customer_orders.empty:
        print(f"Customer with ID {new_order} not found.")
    else:
        customer_orders = customer_orders.sort_values(['date', 'time'], ascending=False)
        recent_order_ids = customer_orders['order_id'].unique()[:2]

        final_recommendations_df = pd.DataFrame(columns=['name', 'ingredients'])

        for order_id in recent_order_ids:
            order_items = customer_orders[customer_orders['order_id'] == order_id]
            #Check customer_id 344
            if len(order_items) == 1:
                item_name = order_items['name'].iloc[0]
                #print(f"Order {order_id} only has 1 item: {item_name}, applying recommend_cb:")
                recommendation1 = recommend_cb(item_name)
                #print(recommendation1)
                existing_names = final_recommendations_df['name'].tolist()
                new_recommendations = recommendation1[~recommendation1['name'].isin(existing_names)]
                final_recommendations_df = pd.concat([final_recommendations_df, new_recommendations], ignore_index=True, axis=0)
            else:
                #print(f"Suggestions for order {order_id}:")
                recommendation2 = recommend_pizza(order_id=order_id,
                                                cosine_sim_colab=cosine_sim_colab,
                                                user_item_matrix=user_item_matrix,
                                                colab_df=colab_df,
                                                top_n=4)
                #print(recommendation2)
                existing_names = final_recommendations_df['name'].tolist()
                new_recommendations = recommendation2[~recommendation2['name'].isin(existing_names)]
                final_recommendations_df = pd.concat([final_recommendations_df, new_recommendations], ignore_index=True, axis=0)
        print("\nYou might like these dishes:")
        print(final_recommendations_df)

        while True:
            option = input("Enter 'more' for more suggestions, or 'done' to exit: ").strip()
            if option.lower() == 'more':
                all_pizzas = df_pitype[['name', 'ingredients']]
                available_pizzas = all_pizzas[~all_pizzas['name'].isin(final_recommendations_df['name'])]
                new_recommendations = available_pizzas
                final_recommendations_df = pd.concat([final_recommendations_df, new_recommendations], ignore_index=True)
                return final_recommendations_df
            elif option.lower() == 'done':
                print("Exiting recommendation process.")
                return final_recommendations_df
            else:
                print("Invalid input. Please enter 'more' or 'done'.")
        return final_recommendations_df

In [27]:
def RS_program():
  #If 'no' is entered, then it is a new customer
  new_order = input("Enter customer ID (or enter 'no' to skip): ").strip()
  if new_order == 'no':
    result = new_customer()
    display(result)
  else:
    result = old_customer(new_order)
    display(result)

In [35]:
RS_program()

Enter customer ID (or enter 'no' to skip): 348
Similar dishes to 'The Classic Deluxe Pizza', you might like:

You might like these dishes:
                                          name  \
0                   The Barbecue Chicken Pizza   
1                        The Five Cheese Pizza   
2                      The Pepper Salami Pizza   
3                  The Italian Capocollo Pizza   
4                              The Greek Pizza   
5                     The Classic Deluxe Pizza   
6                          The Pepperoni Pizza   
7                   The Spinach and Feta Pizza   
8   The Pepperoni, Mushroom, and Peppers Pizza   
9            The Vegetables + Vegetables Pizza   
10                   The Chicken Alfredo Pizza   
11                          The Big Meat Pizza   
12                          The Mexicana Pizza   
13                        The Napolitana Pizza   
14                 The Southwest Chicken Pizza   
15                   The Spinach Supreme Pizza   
16         

,name,ingredients
0,The Barbecue Chicken Pizza,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Go..."
2,The Pepper Salami Pizza,"Genoa Salami, Capocollo, Pepperoni, Tomatoes, ..."
3,The Italian Capocollo Pizza,"Capocollo, Red Peppers, Tomatoes, Goat Cheese,..."
4,The Greek Pizza,"Kalamata Olives, Feta Cheese, Tomatoes, Garlic..."
5,The Classic Deluxe Pizza,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
6,The Pepperoni Pizza,"Mozzarella Cheese, Pepperoni"
7,The Spinach and Feta Pizza,"Spinach, Mushrooms, Red Onions, Feta Cheese, G..."
8,"The Pepperoni, Mushroom, and Peppers Pizza","Pepperoni, Mushrooms, Green Peppers"
9,The Vegetables + Vegetables Pizza,"Mushrooms, Tomatoes, Red Peppers, Green Pepper..."
